In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [2]:
# use the unique() function on the SN column of the data frame
# this will return a list of the uniqure SN values
# use the len() function to determine the length of the list - this will give you the count of unique players 
# save this in a new data frame and display it
userCount = len(purchase_data["SN"].unique())
totalSN = pd.DataFrame({"Total Players": [userCount]})
totalSN

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
# use the unique() function on the Item ID column and the length function to determine
#     the number of unique items in the data frame
uniqueItems = len(purchase_data["Item ID"].unique())

# use the mean() function on the Price column to calculate the average price
avgPrice = purchase_data["Price"].mean()

# use the count() function on the Purchase ID column to determine the total number of purchases
purchaseCount = purchase_data["Purchase ID"].count()

# use the sum() function on the Price column to determine the Total Revenue
totalRev = purchase_data["Price"].sum()

In [4]:
# create a data frame from the calculated values
purchase_analysis = pd.DataFrame({"Number of Unique Items": [uniqueItems],
                                  "Average Price": [avgPrice],
                                  "Number of Purchases": [purchaseCount],
                                  "Total Revenue": [totalRev]
                                 })

# format the Average Price and Total Revenue columns
purchase_analysis["Average Price"] = purchase_analysis["Average Price"].map("${:,.2f}".format)
purchase_analysis["Total Revenue"] = purchase_analysis["Total Revenue"].map("${:,.2f}".format)

# display the summary data frame
purchase_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [5]:
# create a new data frame from the purchased data using only SN, Age, and Gender
gender_df = purchase_data[["SN", "Age", "Gender"]]

# drop the duplicates from the new gender data frame
gender_df = gender_df.drop_duplicates()

In [6]:
# with the new gender data frame (without duplicate values) 
# use loc and count() to find the number of male users, female users, and other/non disclosed
maleCount = gender_df["Gender"].loc[gender_df["Gender"]=="Male"].count()
femaleCount = gender_df["Gender"].loc[gender_df["Gender"]=="Female"].count()
other = userCount - maleCount - femaleCount

# using the previously calculated userCount, calculate the percentage of users by gender
malePct = maleCount/userCount * 100
femalePct = femaleCount/userCount * 100
otherPct = other/userCount * 100

In [7]:
# create a data frame to display the summary of gender demographics
genderAnalysis_df = pd.DataFrame({"Total Count": [maleCount, femaleCount, other],
                         "Percentage of Players": [malePct, femalePct, otherPct]} , 
                                 index =["Male", "Female", "Other / Non-Disclosed"] )

# format the pecentage column
genderAnalysis_df["Percentage of Players"] = genderAnalysis_df["Percentage of Players"].map("{:.2f}%".format)

# display the data frame
genderAnalysis_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [8]:
# Group the purchase data by gender to preform an analysis
genderGroup = purchase_data.groupby("Gender")

# using the grouped data, use the count() function to determine the number of purchases made by gender
groupedPurchasedCount = genderGroup["Purchase ID"].count()

# using the grouped data, use the mean() function to determine the average purchase price by gender
groupedAvgPurchase = genderGroup["Price"].mean()

# using the grouped data, use the sum() function to determine the total value of purchases made by gender
groupedSum = genderGroup["Price"].sum()

# calculate the average total purchase per person using the total value of purchases made by gender
#     and the count of users by gender
ppAvg = groupedSum/genderAnalysis_df["Total Count"]


In [9]:
# create a summary data frame for purchasing analysis by gender
Group_purchaseAnalysis = pd.DataFrame({"Purchase Count": groupedPurchasedCount,
                                       "Average Purchase Price": groupedAvgPurchase,
                                       "Total Purchase Value": groupedSum,
                                       "Avg Total Purchase per Person": ppAvg
                                      })

# format the data frame values
Group_purchaseAnalysis['Average Purchase Price'] = Group_purchaseAnalysis['Average Purchase Price'].map("${:,.2f}".format)
Group_purchaseAnalysis['Total Purchase Value'] = Group_purchaseAnalysis['Total Purchase Value'].map("${:,.2f}".format)
Group_purchaseAnalysis['Avg Total Purchase per Person'] = Group_purchaseAnalysis['Avg Total Purchase per Person'].map("${:,.2f}".format)

# display the data frame
Group_purchaseAnalysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [10]:
# create bins for the following age ranges:
# <10, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40+
# use the max function on the Age column to set the end range for the last age bin 
bins = [0,9,14,19,24,29,34,39,purchase_data["Age"].max()]

# create labels for the bins
group_name = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# use the cut() function from pandas to determine the Age Group for each row in the created gender data frame
# use this data frame so that you are looking at the unique set of users
age_slice = pd.cut(gender_df["Age"], bins, labels = group_name)

# create a new column in the data frame for Age Group
gender_df["Age Group"] = age_slice

In [11]:
# group the data by age group
# and use the count() function to determine the count of players in each age group
totalCount = gender_df.groupby("Age Group")["SN"].count()

# calculate the percentage of players in each age group
pctPlayers = totalCount/userCount *100

# create a new data frame for the Age Demographics data
ageGroup = pd.DataFrame({"Total Count": totalCount, 
                        "Percentage of Players": pctPlayers}, index = group_name)

# format the new data frame
ageGroup['Percentage of Players'] = ageGroup['Percentage of Players'].map("{:.2f}%".format)

# display the data frame
ageGroup

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [12]:
# using the same bins, add the Age Ranges to the purchased data
bins = [0,9,14,19,24,29,34,39,purchase_data["Age"].max()]
group_name = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# create a new data frame from the purchased data frame to work from
age_purchase = purchase_data

# add the Age Range column to the data frame using the cut() function and the bins
age_purchase['Age Ranges'] = pd.cut(age_purchase["Age"], bins, labels = group_name)

In [13]:
# group the new data frame by Age Ranges
purchased_group = age_purchase.groupby('Age Ranges')

# use the count() function to determine the number of purchases by Age Range
purchase_count = purchased_group['Purchase ID'].count()

# use the mean() function to calculate the average purchase price by Age Range
purchase_avg = purchased_group['Price'].mean()

# use the sum() function to calculate the total purchase value by Age Range
purchase_value = purchased_group['Price'].sum()

# calculate the average total purchase per person using the total purchase value by Age Range
#     and the previously calculated values for number of users in each age range
avg_pPerson = purchase_value/ageGroup["Total Count"]

In [14]:
# create a new data frame to store the purchasing analysis data by age
age_analysis = pd.DataFrame({"Purchase Count": purchase_count,
                             "Average Purchase Price": purchase_avg,
                             "Total Purchase Value": purchase_value,
                             "Avg Total Purchase per Person": avg_pPerson
                            }, index = group_name)

# format the new data frame
age_analysis['Average Purchase Price'] = age_analysis['Average Purchase Price'].map('${:,.2f}'.format)
age_analysis['Total Purchase Value'] = age_analysis['Total Purchase Value'].map('${:,.2f}'.format)
age_analysis['Avg Total Purchase per Person'] = age_analysis['Avg Total Purchase per Person'].map('${:,.2f}'.format)
age_analysis.index.name = 'Age Ranges'

# display the data frame
age_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [15]:
# group the purchase data by SN
sn_group = purchase_data.groupby('SN')

# using the grouped data, determine the number of purchases made by each SN using the count() function
sn_purchase_count = sn_group['Purchase ID'].count()

# using the grouped data, determine the average purchase price by SN using the mean() function
sn_avg_purchase = sn_group['Price'].mean()

# using the grouped data, calculate teh total purchase value by SN using the sum() function
sn_total_value = sn_group['Price'].sum()

In [16]:
# create a new data frame to store the calculated values
topSpenders = pd.DataFrame({'Purchase Count': sn_purchase_count,
                           'Average Purchase Price': sn_avg_purchase,
                           'Total Purchase Value': sn_total_value
                           }, index = sn_group.groups.keys())

# sort the data in descending order based on the total purchase value column
topSpenders = topSpenders.sort_values('Total Purchase Value', ascending = False)

# format the data
topSpenders['Average Purchase Price'] = topSpenders['Average Purchase Price'].map('${:,.2f}'.format)
topSpenders['Total Purchase Value'] = topSpenders['Total Purchase Value'].map('${:,.2f}'.format)
topSpenders.index.name = 'SN'

# show a preview of the data frame
topSpenders.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [17]:
# create a data frame from the purchase data using the Item ID, Item Name, and Item Price Columns
items_df = purchase_data[['Item ID', 'Item Name', 'Price']]

In [18]:
# group the data by Item ID and Item Name
items_group = items_df.groupby(['Item ID', 'Item Name'])

# using the grouped data, use the count() function to determin the number of purchases of each item
items_purchase_count = items_group['Price'].count()

# use the mean() function on the grouped data to calculate the Item Price
items_price = items_group['Price'].mean()

# use the sum() function to calculate the Total Purchase Value of each item
items_purchase_value = items_group['Price'].sum()

In [19]:
# create a new data frame to summarize the Most Popular Items
popularItems = pd.DataFrame({'Purchase Count': items_purchase_count,
                            'Item Price': items_price,
                            'Total Purchase Value': items_purchase_value
                            })

# sort data in descending order by purchase count
popularItems = popularItems.sort_values('Purchase Count', ascending = False)

# format data frame
popularItems['Item Price'] = popularItems['Item Price'].map('${:,.2f}'.format)
popularItems['Total Purchase Value'] = popularItems['Total Purchase Value'].map('${:,.2f}'.format)

# display a preview of the data frame
popularItems.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [20]:
# create a data frame using the same data as above
# a new data frame had to be created because columns had been formated and could not be used for sorting
profitableItems = pd.DataFrame({'Purchase Count': items_purchase_count,
                            'Item Price': items_price,
                            'Total Purchase Value': items_purchase_value
                            })

# sort the data in descending order based on the Total Purchase Value
profitableItems = profitableItems.sort_values('Total Purchase Value', ascending = False)

# format the data
profitableItems['Item Price'] = profitableItems['Item Price'].map('${:,.2f}'.format)
profitableItems['Total Purchase Value'] = profitableItems['Total Purchase Value'].map('${:,.2f}'.format)

# display the 5 most proficable items
profitableItems.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
